In [1]:
import math
import struct
import random


In [2]:
def float_to_int_bits(f):
    """Convert a floating-point number to its bit representation as an integer."""
    return int.from_bytes(bytearray(struct.pack('>f', f)), byteorder='big')

# VSAM

In [202]:
# refactor

from math import sqrt
import numpy as np
from collections import Counter as sparse


class hash_family:
    def __init__(self, depth):
        # number of hash functions
        self.depth = depth
        
        # Arrays: F[0] : a1_arr , F[1] : b1_arr , F[2] : a2_arr , F[3] : b2_arr , F[4] : a3_arr , F[5] : b3_arr 
        self.F = np.random.randint(0, 1 << 63 - 1, size=(6, depth), dtype=np.int64)

    @staticmethod
    def hash31(a, b, x):
        r = a * x + b
        
        # int divide by 2^31 (Shift 31) + combine higher order bits with lower
        fold = ((r >> 31) ^ r)
        
        # 2147483647 = 0111...1
        return fold & 2147483647

    def hash(self, x):
        F = self.F
        return self.hash31(F[0], F[1], x)

    def fourwise(self, x):
        F = self.F
        return 2*(((self.hash31(self.hash31(self.hash31(x,F[2],F[3]),x,F[4]),x,F[5])) & 32768)>>15)-1

class sketch:
    def __init__(self, width, depth, hf):
        self.width = width
        self.depth = depth
        self.hf = hf
        self.vec = np.zeros((depth, width))

    def update(self, key, freq=1):
        pos = self.hf.hash(key) % self.width
        delta = self.hf.fourwise(key) * freq
        self.vec[range(self.depth), pos] += delta

    def inner(self, other):
        return np.median(np.einsum('ij,ij->i', self.vec, other.vec))


def make_stream(nkeys, length):
    return np.random.randint(nkeys, size=length)


def make_sparse(S):
    return sparse(S)


def sparse_inner(s1, s2):
    return sum(s1[k] * s2[k] for k in s1 if k in s2)


def create_sketch(width, depth, hf, sp):
    sk = sketch(width, depth, hf)
    for x in sp:
        sk.update(x, sp[x])
    return sk


def create_sketch_for_vector(width, depth, hf, v):
    sk = sketch(width, depth, hf)
    for i, x in enumerate(v):
        sk.update(i, x)
    return sk

""" ------------------------  Testing  -------------------------------- """

def test_sketch_accuracy():
    width = 1500
    depth = 7

    S1 = make_stream(10000, 10000)
    S2 = make_stream(10000, 10000)

    sp1 = make_sparse(S1)
    sp2 = make_sparse(S2)

    hf = hash_family(depth)
    sk1 = create_sketch(width, depth, hf, sp1)
    sk2 = create_sketch(width, depth, hf, sp2)

    inner_product_true = sparse_inner(sp1, sp2)
    inner_product_estimated = sk1.inner(sk2)

    error = abs((inner_product_true - inner_product_estimated) / inner_product_true)
    accuracy = 4 / np.sqrt(width)
    
    print(f"Accuracy: {accuracy}")
    print()
    print(f"True Inner Product: {inner_product_true}, Estimated Inner Product: {inner_product_estimated}")
    print()
    print(f"Error: {error}")
    assert error < accuracy, "Accuracy not sufficient"
    

def test_sketch_accuracy_euclidian_norm(width, depth, n):

    v = np.random.rand(n) # rand vector 0..1 

    hf = hash_family(depth)
    sk = create_sketch_for_vector(width, depth, hf, v)

    inner_product_true = np.inner(v, v)
    inner_product_estimated = sk.inner(sk)

    error = abs((inner_product_true - inner_product_estimated) / inner_product_true)
    accuracy = 4 / np.sqrt(width)

    print(f"Accuracy: {accuracy}")
    print()
    print(f"True Inner Product: {inner_product_true}, Estimated Inner Product: {inner_product_estimated}")
    print()
    print(f"Error: {error}")
    assert error < accuracy, "Accuracy not sufficient"


def cosine_similarity(array1, array2):
    flat_array1 = array1.flatten()
    
    flat_array2 = array2.flatten()
    
    dot_prod = np.dot(flat_array1, flat_array2)
    
    norm1 = np.linalg.norm(flat_array1)
    
    norm2 = np.linalg.norm(flat_array2)
    
    return dot_prod / (norm1 * norm2)
    

def test_sketch_linearity(width, depth, n):
    v1 = np.random.rand(n) # rand vector 0..1 
    
    v2 = np.random.rand(n) # rand vector 0..1 
    
    hf = hash_family(depth)
    
    sk1 = create_sketch_for_vector(width, depth, hf, v1)
    
    sk2 = create_sketch_for_vector(width, depth, hf, v2)
    
    v1_plus_v2 = v1 + v2
    
    sk_1_plus_2 = create_sketch_for_vector(width, depth, hf, v1_plus_v2)
    
    cos_similarity = cosine_similarity(sk1.vec+sk2.vec, sk_1_plus_2.vec)
    
    print(f"Cosine Similarity between sk(v1+v2) and sk(v1)+sk(v2) : {cos_similarity}")
    
    v1_euc_norm = np.inner(v1, v1)
    v1_euc_norm_est = sk1.inner(sk1)
    
    v2_euc_norm = np.inner(v2, v2)
    v2_euc_norm_est = sk2.inner(sk2)
    
    print()
    print(f"||v1||^2 + ||v2||^2 : {v1_euc_norm+v2_euc_norm}")
    print(f"M(sk(v1)) + M(sk(v2)) : {v1_euc_norm_est+v2_euc_norm_est}")
    
    v1_plus_v2_euc_norm = np.inner(v1_plus_v2, v1_plus_v2)
    
    print()
    print(f"||v1 + v2||^2 : {v1_plus_v2_euc_norm}")
    print(f"M(sk(v1) + sk(v2)) : {sk_1_plus_2.inner(sk_1_plus_2)}") # sk_1_plus_2 is similar 100% to sk(v1)+sk(v2)


In [31]:
test_sketch_accuracy_euclidian_norm(width=100, depth=7, n=100)

Accuracy: 0.4

True Inner Product: 34.652949145694066, Estimated Inner Product: 36.375249218185374

Error: 0.04970139959084316


In [204]:
test_sketch_linearity(width=100, depth=7, n=1000000)

Cosine Similarity between sk(v1+v2) and sk(v1)+sk(v2) : 1.0000000000000002

||v1||^2 + ||v2||^2 : 66737.75827192966
M(sk(v1)) + M(sk(v2)) : 71040.97400854882

||v1 + v2||^2 : 116771.86067293465
M(sk(v1) + sk(v2)) : 120027.45356134442


# TENSORFLOW

In [360]:

import nest_asyncio
nest_asyncio.apply()

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
import tensorflow_federated as tff

In [502]:
depth = 7  # number of hash functions
width = 1700  # specifies hash31 : N -> {0, 1, ..., `width`} uniformly.

tf_width = tf.constant(width, dtype=tf.int32)
tf_depth = tf.constant(depth, dtype=tf.int32)

# Pool of three random tuples (A, B) corresponding to a different hash function parameters
# We provide information about pair (F[0], F[1]) , the rest follow this 
# F[0] : shape(depth,) random `a` parameters for each row of the sketch. One row <-> One hash func <-> One `a`
# F[1] : shape(depth,) random `b` parameters for each row of the sketch. One row <-> One hash func <-> One `b`
tf_F = tf.random.uniform(shape=(6, depth), minval=0, maxval=(1 << 31) - 1, dtype=tf.int32)

In [503]:
@tf.function
def sketch_for_vector(v):
    """ Returns AGMS sketch for `v` (vector shape=(n,)). 
    Note: We serialize `F`, `width`, `depth` for efficiency """
    
    F = tf.constant(tf_F)
    width = tf.constant(tf_width)
    depth = tf.constant(tf_depth)

    @tf.function
    def _fourwise(x):
        """ Fourwise independent hash of `x` (int) to {+1, -1}. """
        result = 2 * (tf.bitwise.right_shift(tf.bitwise.bitwise_and(hash31(hash31(hash31(x, F[2], F[3]), x, F[4]), x, F[5]), 32768), 15)) - 1
        return result

    @tf.function
    def _hash31(x, a, b):
        """ _hash31 : N -> {0, 1, ..., width} uniformly """
        r = a * x + b
        fold = tf.bitwise.bitwise_xor(tf.bitwise.right_shift(r, 31), r)
        return tf.bitwise.bitwise_and(fold, 2147483647)

    sketch = tf.zeros(shape=(depth, width), dtype=tf.float32)
    indices = tf.range(tf.shape(v)[0], dtype=tf.int32)

    for i in indices:
        pos = _hash31(i, F[0], F[1]) % width
        delta = tf.cast(_fourwise(i), dtype=tf.float32) * v[i]
        indices_to_update = tf.stack([tf.range(depth, dtype=tf.int32), pos], axis=1)
        sketch = tf.tensor_scatter_nd_add(sketch, indices_to_update, delta)

    return sketch

@tff.tf_computation
def sketch_for_vector_fn(v):
    return sketch_for_vector(v)

In [504]:
@tf.function
def median(v):
    """ Median of tensor `v` with shape=(n,). Note: Suboptimal O(nlogn)"""
    length = tf.shape(v)[0]
    sorted_v = tf.sort(v)
    middle = length // 2

    return tf.cond(
        tf.equal(length % 2, 0),
        lambda: (sorted_v[middle - 1] + sorted_v[middle]) / 2.0,
        lambda: sorted_v[middle]
    )

In [505]:
v = tf.random.uniform(shape=(10000,), minval=0, maxval=10, dtype=tf.float32)

In [506]:
sk = sketch_for_vector(v)

In [507]:
sk

<tf.Tensor: shape=(7, 1700), dtype=float32, numpy=
array([[  1.6201491 ,  -5.1573505 ,  -5.945278  , ...,  -3.3415232 ,
         -9.670412  ,  19.989925  ],
       [  5.9935856 , -11.524099  ,  -5.3829327 , ...,  -3.7804236 ,
         10.356997  ,  16.186014  ],
       [-21.00036   ,   4.3484545 , -26.643091  , ...,  27.928476  ,
        -24.878897  ,   2.739426  ],
       ...,
       [ 13.966475  ,   0.        ,   0.        , ...,   0.        ,
          0.        ,  19.108099  ],
       [  0.76839733, -35.455368  ,   3.0272336 , ...,  19.202957  ,
        -13.429497  ,  10.441992  ],
       [-22.023245  ,  15.0942    , -12.02697   , ...,   6.05863   ,
         -0.8355794 ,   5.2781315 ]], dtype=float32)>

In [508]:
fin = tf.reduce_sum(tf.square(sk), axis=1)

In [509]:
np.inner(v, v)

336741.3

In [510]:
median(fin)

<tf.Tensor: shape=(), dtype=float32, numpy=337869.94>

In [511]:
np.median(fin)

337869.94